## Import

In [14]:
import os
import datetime as dt
import pandas as pd
import numpy as np

In [15]:
# Import custom modules - NEED WINDOWS OS
#from CoEQWAL.imports import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss
import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss

OSError: dlopen(../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll, 0x0006): tried: '../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/python3.9/lib-dynload/../../../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso

## Define file names and ranges for DSS and vars

In [16]:
# list of study names file, tab, range, and extension
DssList = r'../dss_may_2024/Exploratory_Results_v03/callite_expl03_run_listing.xlsx'
DssTab = 'callite_expl03_run_listing'
DssMin = 'A1'
DssMax = 'A601'
DssExt = '_DV.dss'
DssOut = 'EDA_07_01_24_dss_names.csv'

# list of relevant variables file, tab, and range (B & C parts)
VarList = r'../Selected Studies and Variables/May27_variables.xlsx'
VarTab = 'May EDA'
VarMin = 'B1'
VarMax = 'C51'
VarOut = 'EDA_vars_07_01_24.csv'

# launch file path to (or CalSim) studies and output csv name
StudyDir = r'../dss_may_2024/Exploratory_Results_v03'
DataDir = r'../output/extract'
DataOut = 'EDA_data_07_01_24.csv'

# WY csv
WYcsv = 'wytypestable.csv'

# time intervals
begYear = 1921
begMonth = 10
begDay = 31
endYear = 2003
endMonth = 9
endDay = 30

## Read names and construct study name

In [ ]:
dsshdr, dssname = af.read_from_excel(DssList, DssTab, DssMin, DssMax, hdr=True)
dss_names = []
for i in range(len(dssname)):
    dss_names.append(dssname[i][0] + DssExt)
dss_names

## Write dss names to file

In [ ]:
dss_df = (pd.DataFrame(dss_names))
dss_df.to_csv(os.path.join(DataDir, DssOut))

## Read variables

In [ ]:
hdr, vars = af.read_from_excel(VarList, VarTab,VarMin,VarMax,hdr=True)
var_df = pd.DataFrame(data=vars, columns=hdr)
var_df

In [ ]:
var_df.to_csv(os.path.join(DataDir,VarOut))

## Create datasets across studies


In [ ]:
def preprocess_data_dss(df, begYear, begMonth, begDay, endYear, endMonth, endDay, addsl=False, addres = True, addpump = True, adddelcvp = True, adddelcvpag = True, addcvpscex = True, addcvpprf = True, adddelcvpswp = True):
    dvar_list = []
    combined_df = pd.DataFrame()
    TAF_MO_PER_CFS = 60.3707 / 1000

    for i, r in df.iterrows():
        dvar_list.append(f'/{r["DSS Part B"]}/{r["DSS Part C"]}/')

    for dss_name in dss_names:

        print(dss_name)

        # Create a blank python "calsim" object
        thiscs3 = cs3.calsim()

        # add start and end dates
        thiscs3.StartDate = dt.datetime(begYear,begMonth,begDay)
        thiscs3.EndDate = dt.datetime(endYear,endMonth,endDay)

        # add path to DSS (CURRENTLY EXPECTING DSS IN THE BASE DIRECTORY, BUT WE SHOULD MAKE IT A PARAMETER)
        DSS_FP = os.path.join(StudyDir, dss_name)
        thiscs3.DV_FP = DSS_FP

        # Retrieve the DSS data variables from the DSS file
        thiscs3.DVdata = cs3.csDVdata(thiscs3)
        thiscs3.DVdata.getDVts(filter=dvar_list)

        df = thiscs3.DVdata.DVtsDF.copy(deep=True)

        # if storage add the 2 variables to create a new one
        # Add S_SLSCVP and S_SLSWP into S_SLTOT

        if addsl:
            df[('CALLITE', 'S_SLTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_SLCVP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SLSWP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)

        if addpump:
            #df[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_TOTAL_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'D_TOTAL', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addres:
            df[('CALLITE', 'S_RESTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_MELON', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_MLRTN', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'S_RESTOT_NOD', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF') ]].sum(axis=1)

        if adddelcvp:
            #df[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if adddelcvpswp:
            #df[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVPSWP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_SWP_TOTAL', 'DELIVERY-SWP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if adddelcvpag:
            #df[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PAG_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpscex:
            #df[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpprf:
            #df[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PRF_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        new_columns = [(col[0], f'{col[1]}_{dss_name[:-7]}', *col[2:]) if len(col) > 1 else (col[0], '') for col in df.columns]
        df.columns = pd.MultiIndex.from_tuples(new_columns)
        df.columns.names = ['A', 'B', 'C', 'D', 'E', 'F', 'Units']
        combined_df = pd.concat([combined_df, df], axis=1)

    return combined_df

In [ ]:
df = preprocess_data_dss(var_df, begYear, begMonth, begDay, endYear, endMonth, endDay)
df.head(5)

## Write dataframes to file

In [ ]:
df.to_csv(os.path.join(DataDir, DataOut), header=True, index=True)

## End of data extaction